# Glauber on infinite Erdos-Renyi graph
Compute magnetization in the thermodynamic limit $N\to\infty$ for Erdos-Renyi graphs with average connectivity $c=4$ using population dynamics.

In [1]:
using MatrixProductBP, MatrixProductBP.Models
using TensorTrains
using Distributions, Statistics, Base.Threads, Plots, LaTeXStrings
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2;

In [2]:
T = 15      # final time
c = 4.0     # average degree
m⁰ = -0.6   # magnetization at time zero

β = 1.0
J = 0.5
h = 0.0

popsize = 10^2              # population size
svd_trunc = TruncBond(18);  # SVD truncator

In [3]:
wᵢ = fill(HomogeneousGlauberFactor(J, h, β), T+1)
ϕᵢ = [ t == 0 ? [(1+m⁰)/2, (1-m⁰)/2] : ones(2) for t in 0:T]
ψ_neutral = [ones(2,2) for t in 0:T];

In [20]:
function onebpiter_inner(μ, wᵢ, dᵢ, ns, ϕᵢ, ψ, T, svd_trunc)
    C, full, _ = MatrixProductBP.compute_prob_ys(wᵢ, ns, μ, ψ, T, svd_trunc)
    logzᵢ = 0.0
    for j in 1:dᵢ
        B = MatrixProductBP.f_bp_partial_ij(C[j], wᵢ, ϕᵢ, dᵢ - 1, ns, j)
        μ[j] = orthogonalize_right!(mpem2(B); svd_trunc)
        logzᵢ += normalize!(μ[j])
    end
    B = MatrixProductBP.f_bp_partial_i(full, wᵢ, ϕᵢ, dᵢ)
    b = B |> mpem2 |> marginalize
    f = dᵢ == 0 ? 0.0 : -logzᵢ / dᵢ
    return μ, b, f
end

onebpiter_inner (generic function with 1 method)

In [21]:
function iterate_population!(P, bs, btus, wᵢ, degree_prob, ϕᵢ, T, svd_trunc; nsamples=10^2)
    @showprogress for n in 1:nsamples
        d = rand(degree_prob)
        d > popsize && error("Sampled degree $d greater than population size $popsize")
        indices = rand(eachindex(P), d)     
        μ, b, f = onebpiter_inner(P[indices], wᵢ, d, 2, ϕᵢ, fill(ψ_neutral, d), T, svd_trunc)
        P[indices] = μ
        push!(bs, marginals(b))
        push!(btus, twovar_marginals(b))
    end
end

iterate_population! (generic function with 1 method)

In [22]:
degree_prob = Poisson(c)
P = map(1:popsize) do p
    μ = rand_mpem2(2, 2, T)
    normalize!(μ)
    μ
end
bs = Vector{Vector{Float64}}[]
btus =  Matrix{Matrix{Float64}}[];

In [23]:
iterate_population!(P, bs, btus, wᵢ, degree_prob, ϕᵢ, T, svd_trunc; nsamples=3*10^3);

Progress:   0%|▏                                        |  ETA: 1 days, 7:08:01K

InterruptException: InterruptException:

In [8]:
spin(x) = 3 - 2x
# take the average beliefs over the last `ns` iterations
ns = 1000
range = length(btus)+1-min(ns, length(btus)):length(btus)
ms = [expectation.(spin, b) for b in bs[range]]
m_avg = mean(ms)
m_std = std(ms) ./ sqrt(length(ms));

MethodError: MethodError: no method matching zero(::Type{Vector{Float64}})
The function `zero` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{Dates.Date})
   @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/types.jl:459
  zero(!Matched::Type{IndexedGraphs.NullNumber})
   @ IndexedGraphs ~/.julia/packages/IndexedGraphs/rBQgK/src/utils.jl:59
  ...


In [9]:
spin(x) = 3 - 2x
rs = [expectation.(spin, btu) for btu in btus[range]]
cs = MatrixProductBP.covariance.(rs, ms)
c_avg = mean(cs)
c_std = std(cs) ./ sqrt(length(cs));

MethodError: MethodError: no method matching zero(::Type{Matrix{Float64}})
The function `zero` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{Dates.Date})
   @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/types.jl:459
  zero(!Matched::Type{IndexedGraphs.NullNumber})
   @ IndexedGraphs ~/.julia/packages/IndexedGraphs/rBQgK/src/utils.jl:59
  ...


In [10]:
plot_autocovs(args...; kw...) = plot_autocovs!(plot(), args...; kw...)
function plot_autocovs!(pl, c; skip_even=false, T=size(c,1)-1, us = 0:(2*floor(Int, T/10)):T, kw...)
    step = skip_even ? 2 : 1
    cg = cgrad(:matter, length(us)+1, categorical=true)
    ylabel = L"\langle x_{i}^t x_{i}^{t+\Delta t}\rangle-" * 
        L"\langle x_{i}^t\rangle\langle x_{i}^{t+\Delta t}\rangle"
    xlabel = L"\Delta t"
    plot!(pl; xlabel, ylabel, legend=:outertopright)
     
    for (j,u) in enumerate(us)
        plot!(pl, u:-step:1, c[(0:step:u-1) .+ 1,u+1], label="t+Δt=$u", c=cg[j+1], msc=:auto; kw...) 
    end
    pl
end 

plot_autocovs(c_avg; skip_even=true, yaxis=:log10, m=:o, us=4:2:T)

UndefVarError: UndefVarError: `c_avg` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [11]:
using Graphs, IndexedGraphs

seed = 1
N = 5*10^3
g = erdos_renyi(N, c/N; seed)
ising = Ising(IndexedGraph(g); J=fill(J, ne(g)), h=fill(h, N), β)
bp = mpbp(Glauber(ising, T); ϕ = fill(ϕᵢ, N))
sms = SoftMarginSampler(bp);

In [12]:
sample!(sms, 5*10^3)
m_mc = [vec(spin.(mean(X, dims=1))) for X in sms.X];

SoftMargin sampling 100%|████████████████████████████████| Time: 0:08:08


In [13]:
m_eq = equilibrium_magnetization(ErdosRenyi(c); pJ=Dirac(J), β, ph=Dirac(h), popsize=2*10^6,
        maxiter=10^2, tol=1e-6, nsamples=10^4);

Running PopDyn: iter 2    Time: 0:00:06

Running PopDyn: iter 3    Time: 0:00:08

Running PopDyn: iter 4    Time: 0:00:11

Running PopDyn: iter 5    Time: 0:00:13

Running PopDyn: iter 6    Time: 0:00:15

Running PopDyn: iter 7    Time: 0:00:18

Running PopDyn: iter 8    Time: 0:00:20

Running PopDyn: iter 9    Time: 0:00:22

Running PopDyn: iter 10    Time: 0:00:25

Running PopDyn: iter 11    Time: 0:00:27

Running PopDyn: iter 12    Time: 0:00:29

Running PopDyn: iter 13    Time: 0:00:31

Running PopDyn: iter 14    Time: 0:00:34

Running PopDyn: iter 15    Time: 0:00:36

Running PopDyn: iter 16    Time: 0:00:38

Running PopDyn: iter 17    Time: 0:00:41

Running PopDyn: iter 18    Time: 0:00:43

Running PopDyn: iter 19    Time: 0:00:46

Running PopDyn: iter 20    Time: 0:00:48

Running PopDyn: iter 21    Time: 0:00:50

Running PopDyn: iter 22    Time: 0:00:53

Running PopDyn: iter 23    Time: 0:00:55

Running PopDyn: iter 24    Time: 0:00:57

Running PopDyn: iter 25    Time: 0:01:00



In [14]:
pl = plot(0:T, m_avg, lw=3, ribbon=m_std, xlabel="time", ylabel="magnetiz",
    title="Glauber on infinite Erdos-Renyi with average degree $(Int(c))", titlefontsize=12,
    size = (600,300), label="MPBP")
pl2 = deepcopy(pl)
scatter!(pl2, 0:T, mean(m_mc), label="MC", c=:black, m=:diamond)
pl3 = deepcopy(pl2)
hline!(pl3, [-m_eq.val], label="equilibrium", c=:red, ls=:dash, lw=1, size=(600,300)#=, ribbon=[-m_eq.err]=#)

UndefVarError: UndefVarError: `m_std` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
# jldsave("plot_data/glauber_erdos_renyi.jld2"; T, m_mc, m_avg, m_std, m_eq, c_avg, c_std);